In [120]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib

Using matplotlib backend: Qt5Agg


In [114]:
# let's define the different wavelengths for the different files.
# I use this in order to open the right files
folder_path = '../data/adas/'
wavelengths = ['4101.2', '4339.9', '4860.6', '6561.9']

# try to read density and temperature from different files
n_e = pd.read_csv(folder_path + 'density_de.txt', delim_whitespace=True,
                 squeeze=True, header=None)
T_e = pd.read_csv(folder_path + 'temperature_de.txt', delim_whitespace=True,
                 squeeze=True, header=None)
n_e = n_e.T
T_e = T_e.T
n_e.columns = ['n_e']
T_e.columns = ['T_e']

In [143]:
# we should read the number of lines for every file, but I hope
# every file has the same lines number. In case this won't be true,
# this function has to be changed with range(0, 96) to range(0, num_lines)
df = pd.DataFrame()
for wavelength in wavelengths:
    
    file_path = folder_path + wavelength + '.txt'
    # read four dataframes because the files is splitted across 4 lines
    df0 = pd.read_csv(file_path, delim_whitespace=True,
                     skiprows=[x for x in range(0, 96) if (x+4)%4 != 0],
                     header=None)
    df1 = pd.read_csv(file_path, delim_whitespace=True,
                     skiprows=[x for x in range(0, 96) if (x+3)%4 != 0],
                     header=None)
    df2 = pd.read_csv(file_path, delim_whitespace=True,
                     skiprows=[x for x in range(0, 96) if (x+2)%4 != 0],
                     header=None)
    df3 = pd.read_csv(file_path, delim_whitespace=True,
                     skiprows=[x for x in range(0, 96) if (x+1)%4 != 0],
                     header=None)
    df_wavelength = pd.concat([df0, df1, df2, df3], axis=1, ignore_index=True)
    df_wavelength = df_wavelength.set_index(n_e['n_e'])
    df_wavelength.columns = T_e['T_e']
    df_wavelength['wavelength'] = wavelength
    df_wavelength['n_e'] = df_wavelength.index.values
    df_melt = pd.melt(df_wavelength, id_vars=['n_e', 'wavelength'], var_name='T_e',
                     value_name='X_e')
    df = df.append(df_melt)

In [183]:
a = df.groupby('wavelength', as_index=False)
# afaik h-alpha = 6561.9
# h-beta 4860.6
# h-gamma 4339.9
# h-delta 4101.2
# let's create a dataframe with the following columns:
# | wl1 | wl2 | X1 | X2 | Te | ne | ratio |
# the ration will be the respect to the following:
# ha/hb ---- ha/hg ----- ha/hd
ha = wavelengths[3]
hb = wavelengths[2]
hg = wavelengths[1]
hd = wavelengths[0]

df_ha = a.get_group(wavelengths[3])
df_hb = a.get_group(wavelengths[2])
df_hg = a.get_group(wavelengths[1])
df_hd = a.get_group(wavelengths[0])
# create a new dataframe for ratios
# merge df_ha with df_hb, df_ha with df_hg and so on
df_ratio = pd.DataFrame()
df_ratio = df_ratio.append(pd.merge(df_ha, df_hb, on=['n_e', 'T_e'], 
                                    suffixes=('_1', '_2')))
df_ratio = df_ratio.append(pd.merge(df_ha, df_hg, on=['n_e', 'T_e'], 
                                    suffixes=('_1', '_2')))
df_ratio = df_ratio.append(pd.merge(df_ha, df_hd, on=['n_e', 'T_e'], 
                                    suffixes=('_1', '_2')))
# calculate the ratio between pecs
df_ratio['ratio'] = df_ratio.X_e_1 / df_ratio.X_e_2
# group by wavelength ratio
group_ratio = df_ratio.groupby(['wavelength_1', 'wavelength_2'])

In [232]:
group_b = group_ratio.get_group((ha, hb))
group_g = group_ratio.get_group((ha, hg))
group_d = group_ratio.get_group((ha, hd))

# let's try to plot group d
#fig = plt.figure()
#ax = fig.add_subplot()
group_d_T = group_d[group_d.T_e < 20].groupby('n_e')
for name, group in group_d_T:
    if 1e10 <= float(name) < 1e12:
        plt.plot(group.T_e, group.ratio, 
            label='n = {:.3e}'.format(float(name)))
plt.xlabel('T (eV)')
plt.ylabel('ratio')
plt.legend()

In [226]:
# let's try to plot group d but with respect to densities
#fig = plt.figure()
#ax = fig.add_subplot()
group_d_n = group_d[group_d.n_e < 1e13].groupby('T_e')
for name, group in group_d_n:
    if float(name) < 20:
        plt.plot(group.n_e, group.ratio, 
            label='T = {:.3e}'.format(float(name)))
plt.xlabel('n ($cm^{3}$)')
plt.ylabel('ratio')
plt.legend()

In [230]:
plt.clf()